In [1]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from matplotlib import image
import os

In [2]:
# load images of size 112,92 

train_image_load = []
test_image_load = []

total_person = 40
train_size = 6
test_size = 4

# tracks train set and test set

for i in range(1,total_person+1):
    j = 1
    arr = os.listdir('s'+str(i)+'/')
    for _ in arr:
        if(j <= train_size):
            train_image_load.append((image.imread('s'+str(i)+'/'+_)).flatten())
        else:
            test_image_load.append((image.imread('s'+str(i)+'/'+_)).flatten())
        j+=1

In [3]:
def mean_face(Face):
    mn,p = Face.shape
    M = np.zeros((mn,1))
    for i in range(mn):
        for j in range(p):
            M[i] += Face[i][j]
        M[i] /= p
    return M

In [4]:
def surrogate_cov(X):
# Returns covariance of X
    rows,cols = X.shape 
    cov = np.zeros((cols,cols))
     
    # Covariances        
    for z in range(cols):
        for y in range(cols):
                cov[z][y] = np.dot(X[:,y].T,X[:,z])/rows                
    return cov 

In [64]:
def variance(X):
    var = X
    n,c = X.shape
    M = mean_face(X)
    
    for i in range(n):
        for j in range(c):
            var[i][j] = var[i][j] - M[i]
    
    return var
    

### Step 1 Apply PCA

In [5]:

# step 1 generate face dataset (mn*p)
# 60% for training (6 images/folder)
Face_Db = np.array(train_image_load).T  
mn,p = Face_Db.shape

# step 2 calculate mean (mn*1)
M = mean_face(Face_Db)

# step 3 mean 0
delta = np.zeros((Face_Db.shape))
for i in range(mn):
    for j in range(p):
        delta[i][j] = Face_Db[i][j] - M[i]
        
# step 4 surrogate covariance calculation
cov = surrogate_cov(delta)   

# step 5 eigen values and eigen vectors (sorted decreasing order) of cov
eigenValues, eigenVectors = np.linalg.eig(cov)
idx = eigenValues.argsort()[::-1]   
eig_val = eigenValues[idx]
eig_vec = eigenVectors[:,idx]

# setting k
k = 5

# Psi
psi = eig_vec[:,0:k]

# step 7 Generate Eigen Faces
phi = np.dot(psi.T,delta.T)

PF = np.dot(phi,delta)


### Step 2 Divide data into classes

In [6]:
n = train_size
number_of_classes = p//n

### Step 3 Find means of each class and mean of Projected Faces

In [51]:
mean_each_class = np.zeros((k,number_of_classes))

old = 0
new = n
for i in range(number_of_classes):
    mean_each_class[:,i] = mean_face(PF[:,old:new]).T
    old = new
    new = new + n
    
mean_PF = mean_face(PF)

In [15]:
mean_each_class.shape

(5, 40)

In [26]:
mean_PF.shape

(5, 1)

### Step 4: Within Class Scatter Matrix and Between Class Scatter Matrix

In [70]:
# Within Class Scatter Matrix
SW = []
old = 0
new = n
for i in range(number_of_classes):
    V_mu = variance(PF[:,old:new])
    SW.append(np.dot(V_mu,V_mu.T))
    old = new
    new = new + n

SW = np.array(SW)    
SW.shape

(40, 5, 5)

In [72]:
# Between Class Scatter Matrix